In [1]:
import pandas as pd
import re
import nltk
from collections import defaultdict # Dictionaries with default values
import string


In [2]:
## importing data sets and dropping nan's
df = pd.read_csv('GFM_data.csv',sep = '\t')
df = df.loc[df['Text'].dropna().index]
df.head()

,Unnamed: 0,Url,Category,Position,Title,Location,Amount_Raised,Goal,Number_of_Donations,Length_of_Fundraising,FB_Shares,Number_of_Donors,Followers,Text
0,0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0,Justice for Jacob Blake,"Kenosha, WI",2296580.0,3000000.0,72.9K,72 days 00:33:52.072310000,118K,72.5K,73.4K,On August 23rd my son was shot multiple times ...
1,0,https://www.gofundme.com/f/official-navajo-nat...,Medical,0,Official Navajo Nation COVID-19 Relief Fund,"Window Rock, AZ",1831590.0,1000000.0,22K,184 days 00:33:53.278684000,71.3K,21.5K,21.6K,\nThe Navajo Nation COVID-19 Fund has been est...
2,0,https://www.gofundme.com/f/Tommy-Rivers-Rest-Up,Medical,0,"Rest up, Tommy, we'll see you soon","Raised $105,973 from 1795 donations",652554.0,1000000.0,11K,110 days 00:33:55.164400000,20.8K,10.1K,10.2K,"First, thank you for being here. Tommy Rivers ..."
3,0,https://www.gofundme.com/f/brandon039s-medical...,Medical,1,OFFICIAL BRANDON SAENZ MEDICAL FUND,"Tyler, TX",570299.0,750000.0,24.7K,154 days 00:33:56.151397000,5.5K,24.3K,24.5K,My name is Melissa Green and I am the mother o...
4,0,https://www.gofundme.com/f/please-help-me-in-t...,Medical,1,PLEASE HELP ME IN THE BATTLE FOR MY LIFE,"New Milford, NJ",498256.0,600000.0,7.8K,20 days 00:33:59.899292000,6.6K,7.7K,7.9K,PLEASE HELP ME IN THE BATTLE FOR MY LIFE. My n...


In [23]:
df['Text'][6]

"Mattress Mack invites you to help him SAVE CAMP HOPE by donating to this campaign today! The PTSD Foundation of America's Camp Hope is a facility located in Houston, TX that offers temporary housing at no cost to our nation's veterans and their families as veterans find help for combat related Post Traumatic Stress Disorder (PTSD). In addition to temporary housing, Camp Hope offers veterans\xa0intensive peer support and mentoring programs that heal and provide hope in their civilian lives.Camp Hope is currently at risk of shutting down and must raise $1,000,000 to keep their doors open to America's veterans. Your generous donation will help to SAVE CAMP HOPE so that they may continue to offer life saving services to those who have so selflessly served the United States of America.\xa0Please donate to Mattress Mack's campaign to SAVE CAMP HOPE today, and be a hero to our nation's heroes. \xa0To learn more about Camp Hope, please visit https://bit.ly/3gPAA4j.Executive Director of Camp H

In [5]:
punct = string.punctuation
wordCount = defaultdict(int)
totalWords = 0

In [6]:
tokens_list = []
for t in df['Text']:
    t = t.lower() # lowercase string
    t = re.sub('https*://[^\s]*','',t) ## urls
    t = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}', '', t) ## replacing phone numbers
    t = re.sub('\d+[ \n\r\f\v][A-Za-z0-9]+\s[A-Za-z0-9]*[ \r\f\v]*[A-Za-z0-9]*','', t) ## addresses
    t = re.sub('[\w]+@\w*\.\w*\.*\w*','', t) ## replacing email addresses
    t = [c for c in t if not (c in punct)] # non-punct characters
    t = ''.join(t) # convert back to string
    words = t.strip().split() # tokenizes
    tokens_list.append(words)
    for w in words:
        #w = stemmer.stem(w)
        totalWords += 1
        wordCount[w] += 1


In [7]:
df['tokens'] = tokens_list

In [25]:
df['tokens'][7]

['needs',
 'a',
 'life',
 'saving',
 'drug',
 'that',
 'costs',
 '21',
 'million',
 'follow',
 'rhys’s',
 'story',
 'and',
 'current',
 'updates',
 'at',
 'wwwriseupforrhysorg',
 'and',
 'on',
 'instagram',
 'riseupforrhysin',
 'may',
 'of',
 'in',
 'scotland',
 'rory',
 'and',
 'i',
 'found',
 'out',
 'we',
 'were',
 'pregnant',
 'though',
 'i',
 'already',
 'knew',
 'mother’s',
 'intuition',
 'kicks',
 'in',
 'early',
 'our',
 'baby',
 'girl',
 'was',
 'due',
 'on',
 'christmas',
 'day',
 'and',
 'after',
 'a',
 'perfectly',
 'healthy',
 'pregnancy',
 'and',
 'we',
 'welcomed',
 'her',
 'december',
 '4',
 '13pm',
 'even',
 'though',
 'i',
 'had',
 'sworn',
 'not',
 'to',
 'give',
 'my',
 'children',
 'confusingtopronounce',
 'welsh',
 'names',
 'as',
 'my',
 'mother',
 'had',
 'done',
 'to',
 'me',
 'we',
 'decided',
 'to',
 'name',
 'her',
 'rhys',
 'pronounced',
 'reese',
 'overnight',
 'she',
 'made',
 'us',
 'a',
 'family',
 'and',
 'even',
 'her',
 'fur',
 'brother',
 'bruce',
 

'bicent@gmail.com'